# Convergence

> Scripts to perform the refinement on the generated orbits

In [ ]:
#| default_exp convergence

In [ ]:
#| export
#| hide
from julia.api import Julia
from typing import Optional, List, Tuple
jl = Julia(compiled_modules=False)
from julia import Main
import logging
import numpy as np
import pandas as pd
import os
from orbit_generation.path import get_julia_file_path

In [ ]:
#| export
#| hide
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Julia Wrapper

In [ ]:
#| export
julia_path = get_julia_file_path('convergence_algorithm.jl')
Main.include(julia_path)

def differential_correction(orbit: np.ndarray,  # Orbit data with shape [num_timesteps, 7]
                          μ: float,  # Gravitational parameter
                          variable_time: bool = True,  # Whether to use variable time nodes
                          time_flight: Optional[float] = None,  # Total time of flight
                          jacobi_constant: Optional[float] = None,  # Jacobi constant
                          X_end: Optional[np.ndarray] = None,  # Terminal state vector (shape: [6])
                          tol: float = 1e-9,  # Tolerance for convergence
                          max_iter: int = 20,  # Maximum number of iterations
                          printout: bool = False,  # Whether to print iteration logs
                          DX_0: Optional[np.ndarray] = None,  # Initial guess for state vector correction
                          X_big_0: Optional[np.ndarray] = None,  # Auxiliary initial guess
                          δ: Optional[float] = None,  # Step size or perturbation parameter
                          ) -> Tuple[np.ndarray, np.ndarray, float, int, int]:
    """
    Performs differential correction on an orbit using Julia implementation.
    """
    # Convert Python None to Julia 'nothing' using an empty list
    julia_time_flight = [] if time_flight is None else float(time_flight)
    julia_jacobi_constant = [] if jacobi_constant is None else float(jacobi_constant)
    julia_X_end = [] if X_end is None else Main.Vector(X_end.tolist())
    julia_DX_0 = [] if DX_0 is None else Main.Vector(DX_0.tolist())
    julia_X_big_0 = [] if X_big_0 is None else Main.Vector(X_big_0.tolist())
    julia_δ = [] if δ is None else float(δ)

    # Convert NumPy array to Julia array
    julia_orbit = Main.Matrix(orbit.tolist())

    # Call the Julia function
    result = Main.differential_correction(
        julia_orbit,
        μ,
        variable_time=variable_time,
        time_flight=julia_time_flight,
        jacobi_constant=julia_jacobi_constant,
        X_end=julia_X_end,
        tol=tol,
        max_iter=max_iter,
        printout=printout,
        DX_0=julia_DX_0,
        X_big_0=julia_X_big_0,
        δ=julia_δ
    )

    # Extract results from Julia tuples
    # Assuming the result is a tuple: (X_corrected, t_corrected, norm, iterations, success)
    X_corrected = np.array(result[0]).reshape(-1, 6)
    t_vec_corrected = np.array(result[1]).flatten()
    norm_F_or_G = float(result[2])
    iterations = int(result[3])
    success = int(result[4])

    return X_corrected, t_vec_corrected, norm_F_or_G, iterations, success

In [ ]:
#| export
def create_converged_orbits_df(converged_indices: List[int],  # List of orbit indices that have converged
                              orbit_array: np.ndarray,  # Original array containing all orbit data
                              converged_orbits: np.ndarray,  # Array containing corrected converged orbits 
                              errors: np.ndarray,  # Array of norm values for each converged orbit
                              iterations: np.ndarray,  # Array of iteration counts for each converged orbit
                              ) -> pd.DataFrame:
    """
    Creates a DataFrame containing detailed information about converged orbits.
    """
    data = []
    for idx, orbit_index in enumerate(converged_indices):
        initial_orbit = orbit_array[orbit_index]
        corrected_orbit = converged_orbits[idx]
        
        # Extract old and new periods
        old_period = initial_orbit[-1, 0]
        new_period = corrected_orbit[-1, 0]
        
        # Extract initial state from the original orbit
        initial_state = initial_orbit[0, 1:7]  # [pos_x0, pos_y0, pos_z0, vel_x0, vel_y0, vel_z0]
        
        # Extract initial state from the corrected orbit
        final_state = corrected_orbit[0, 1:7]
        
        # Compile the data row
        row = {
            'id': orbit_index,
            'old_period': old_period,
            'new_period': new_period,
            'iterations_convergence': iterations[idx],
            'error': errors[idx],
            'initial_pos_x0': initial_state[0],
            'initial_pos_y0': initial_state[1],
            'initial_pos_z0': initial_state[2],
            'initial_vel_x0': initial_state[3],
            'initial_vel_y0': initial_state[4],
            'initial_vel_z0': initial_state[5],
            'final_pos_x0': final_state[0],
            'final_pos_y0': final_state[1],
            'final_pos_z0': final_state[2],
            'final_vel_x0': final_state[3],
            'final_vel_y0': final_state[4],
            'final_vel_z0': final_state[5]
        }
        data.append(row)
    
    df = pd.DataFrame(data)
    return df

In [ ]:
#| export
def process_diferential_correction_orbits(
    orbit_array: np.ndarray,  # Array containing orbit data with shape [num_orbits, num_timesteps, 7]. First element in last dimension is time
    μ: float,  # Gravitational parameter
    variable_time: bool = True,  # Whether to use variable time nodes for correction
    tol: float = 1e-9,  # Tolerance for convergence in differential correction
    max_iter: int = 20,  # Maximum number of iterations for differential correction
    printout: bool = False,  # Whether to print iteration logs
    ) -> tuple[np.ndarray, pd.DataFrame]:
    """
    Processes a set of orbits by providing the orbit array directly to differential correction.
    """
    # Lists to store results
    converged_orbits_list = []
    converged_indices = []
    errors_list = []
    iterations_list = []
    
    num_orbits, num_timesteps, data_length = orbit_array.shape
    
    # Validate data_length
    if data_length != 7:
        logger.error("Data length is incorrect. Expected 7 elements (1 time + 6 state components).")
        raise ValueError("Data length is incorrect. Expected 7 elements (1 time + 6 state components).")
    
    # Loop over each orbit
    for i in range(num_orbits):
        logger.info(f"Processing orbit {i+1}/{num_orbits}")
        
        # Extract each orbit
        orbit = orbit_array[i]  # Shape: [num_timesteps, 7]

        # Apply differential correction
        try:
            X_corrected, t_corrected, norm_val, iterations, success_flag = differential_correction(
                orbit=orbit, 
                μ=μ, 
                variable_time=variable_time, 
                tol=tol, 
                max_iter=float(max_iter), 
                printout=printout
            )
        except Exception as e:
            logger.error(f"Orbit {i}: Differential correction failed with error: {e}")
            continue

        logger.info(f"Orbit {i}: Success={success_flag}, Norm={norm_val}, Iterations={iterations}")
                
        # Check the success flag
        if success_flag == 1:
            # If successful, store the corrected orbit and additional info
            corrected_orbit = np.hstack((t_corrected[:, np.newaxis], X_corrected))
            converged_orbits_list.append(corrected_orbit)
            errors_list.append(norm_val)
            iterations_list.append(iterations)
            converged_indices.append(i)
        else:
            # Orbit failed to converge; no action needed since we're not tracking failures
            continue

    # Convert lists to NumPy arrays
    if converged_orbits_list:
        converged_orbits_array = np.array(converged_orbits_list)
        errors_array = np.array(errors_list)
        iterations_array = np.array(iterations_list)
    else:
        converged_orbits_array = np.empty((0, num_timesteps, 7))
        errors_array = np.array([])
        iterations_array = np.array([])
    
    # Create DataFrame for converged orbits
    if converged_orbits_list:
        converged_orbits_df = create_converged_orbits_df(
            converged_indices=converged_indices,
            orbit_array=orbit_array,
            converged_orbits=converged_orbits_array,
            errors=errors_array,
            iterations=iterations_array
        )
    else:
        converged_orbits_df = pd.DataFrame(columns=[
            'id', 'old_period', 'new_period', 'iterations_convergence', 'error',
            'initial_pos_x0', 'initial_pos_y0', 'initial_pos_z0',
            'initial_vel_x0', 'initial_vel_y0', 'initial_vel_z0',
            'final_pos_x0', 'final_pos_y0', 'final_pos_z0',
            'final_vel_x0', 'final_vel_y0', 'final_vel_z0'
        ])
    
    return converged_orbits_array, converged_orbits_df

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()